In [73]:
import pandas  as pd 
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
from datetime import datetime as dt 
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [74]:
traffic = pd.read_csv('traffic_train.csv')
traffic.head(10)
traffic['date_time'] = pd.to_datetime(traffic['date_time'])
traffic['day'] = traffic.date_time.dt.day
traffic.head()

,date_time,weather_general,weather_detailed,clouds_coverage_pct,temperature,rain_mm,snow_mm,traffic,day
0,2014-10-01 00:00:00,Clear,sky is clear,1,11.5,0.0,0.0,508,1
1,2014-10-01 01:00:00,Clear,sky is clear,1,10.3,0.0,0.0,323,1
2,2014-10-01 02:00:00,Clear,sky is clear,1,8.0,0.0,0.0,274,1
3,2014-10-01 03:00:00,Clear,sky is clear,1,7.9,0.0,0.0,372,1
4,2014-10-01 04:00:00,Clear,sky is clear,1,6.4,0.0,0.0,812,1


In [75]:
def dayFromDate(traffic):
    traffic['date_time'] = pd.to_datetime(traffic['date_time'])
    traffic['hour'] = traffic.date_time.dt.hour
    traffic['hour_interval'] = pd.cut(traffic.hour, bins = [-1,6,12,18,24], labels = ['0-6','6-12', '12-18', '18-24'])
    traffic['day'] = traffic.date_time.dt.day_of_week  
    traffic['month'] = traffic.date_time.dt.month
    traffic['year'] = traffic.date_time.dt.year
    return(traffic)

dayFromDate(traffic)


,date_time,weather_general,weather_detailed,clouds_coverage_pct,temperature,rain_mm,snow_mm,traffic,day,hour,hour_interval,month,year
0,2014-10-01 00:00:00,Clear,sky is clear,1,11.5,0.0,0.0,508,2,0,0-6,10,2014
1,2014-10-01 01:00:00,Clear,sky is clear,1,10.3,0.0,0.0,323,2,1,0-6,10,2014
2,2014-10-01 02:00:00,Clear,sky is clear,1,8.0,0.0,0.0,274,2,2,0-6,10,2014
3,2014-10-01 03:00:00,Clear,sky is clear,1,7.9,0.0,0.0,372,2,3,0-6,10,2014
4,2014-10-01 04:00:00,Clear,sky is clear,1,6.4,0.0,0.0,812,2,4,0-6,10,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29696,2018-12-31 22:00:00,Mist,mist,90,-0.2,0.0,0.0,1228,0,22,18-24,12,2018
29697,2018-12-31 22:00:00,Snow,light snow,90,-0.2,0.0,0.0,1218,0,22,18-24,12,2018
29698,2018-12-31 23:00:00,Mist,mist,90,-0.1,0.0,0.0,830,0,23,18-24,12,2018
29699,2018-12-31 23:00:00,Snow,light snow,90,-0.1,0.0,0.0,828,0,23,18-24,12,2018


In [33]:
traffic.info()
traffic.weather_detailed.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29701 entries, 0 to 29700
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date_time            29701 non-null  datetime64[ns]
 1   weather_general      29701 non-null  object        
 2   weather_detailed     29701 non-null  object        
 3   clouds_coverage_pct  29701 non-null  int64         
 4   temperature          29701 non-null  float64       
 5   rain_mm              29701 non-null  float64       
 6   snow_mm              29701 non-null  float64       
 7   traffic              29701 non-null  int64         
 8   day                  29701 non-null  int64         
 9   month                29701 non-null  int64         
 10  hour                 29701 non-null  int64         
 11  year                 29701 non-null  int64         
 12  hour_interval        29701 non-null  category      
dtypes: category(1), datetime64[ns](

sky is clear                           8030
mist                                   3491
overcast clouds                        3491
broken clouds                          3130
light rain                             2248
scattered clouds                       2044
few clouds                             1499
light snow                             1046
moderate rain                           966
haze                                    743
heavy snow                              581
light intensity drizzle                 551
fog                                     481
drizzle                                 337
proximity thunderstorm                  311
heavy intensity rain                    287
snow                                    157
thunderstorm                             58
thunderstorm with heavy rain             42
thunderstorm with light rain             32
heavy intensity drizzle                  31
proximity shower rain                    31
thunderstorm with rain          

In [76]:
def ordinal_whether(traffic):
    traffic['ordinal_weather'] = np.where(traffic.weather_detailed.isin(['fog', 'freezing rain', 'heavy intensity rain','heavy snow','proximity thunderstorm',
                                 'proximity thunderstorm with drizzle', 'proximity thunderstorm with rain', 'sleet', 'squalls',
                                 'thunderstorm', 'thunderstorm with drizzle', 'thunderstorm with heavy rain', 'thunderstorm with light drizzle',
                                  'thunderstorm with light rain', 'thunderstorm with rain', 'very heavy rain']), 0, #fatalne
                                 np.where(traffic.weather_detailed.isin(['drizzle','haze', 'heavy intensity drizzle','light intensity drizzle','light intensity shower rain',
                                 'light rain', 'light rain and snow', 'light shower snow', 'light snow','mist', 'moderate rain', 'shower drizzle',
                                  'shower snow', 'smoke', 'snow', ]), 1, #kiepskie
                                 np.where(traffic.weather_detailed.isin(['broken clouds','few clouds', 'overcast clouds', 'proximity shower rain',
                                 ]), 2, #umiarkowane
                                 np.where(traffic.weather_detailed.isin(['scattered clouds', 'sky is clear', ]), 3, 99 )))) #dobre
    return (traffic)

In [78]:
def weather_dummies(traffic):
    t1 = pd.get_dummies(traffic['weather_general'])
    t1['tmp'] = range(0,len(traffic.weather_detailed))
    traffic['tmp'] = range(0,len(traffic.weather_detailed))
    traffic = pd.merge(traffic, t1, on = ['tmp']).drop('tmp', axis = 1)
    return(traffic)


Clouds          10164
Clear            8030
Rain             3560
Mist             3491
Snow             1794
Drizzle           920
Haze              743
Thunderstorm      497
Fog               481
Smoke              17
Squall              4
Name: weather_general, dtype: int64